# SYB MVP Tutorial

This notebook demonstrates how to use the MVP (Minimum Viable Product) version of the Sybil Resistant System. This tool is meant as an interactive demonstration of the MVP version of the [SYB project](https://syb.tokamak.network) and its [Network Explorer](https://syb.tokamak.network/explorer).

## Smart Contract and Scoring algorithm

The implementation is based on the [deployed smart contract](https://sepolia.etherscan.io/address/0x02Cb439549AED1A6c8334430A1D5d320685c3E62#code). The details of the **scoring algorithm** logic can be found in the [MVP Scoring Algorithm Specification Document](https://www.notion.so/tokamak/SYB-MVP-Algorithm-specification-29cd96a400a380c289c0e15aa2ad242f?source=copy_link).

## Getting Started: The MVP Contract (VouchMinimal)

Let's set up and run the MVP interface!

### Step 1: Import Required Modules and create a random network.

We import the needed modules and create a random network consisting of 9 users. The first 8 users represent the initial network, while the 9th, 'User', is the user's node.

In [ ]:
# Import the MVP components
from syb_mvp_ui import create_random_mvp_network, SYBMvpUserInterface
from contract_interface_mvp import VouchMinimal, generate_mul_eth_addresses, Node
 
# Create a network with 8 users
contract, users = create_random_mvp_network(num_users=8)

# --- Create a 9th node to be our main 'User' ---
new_user_addr = generate_mul_eth_addresses(1)[0]
new_user_name = "User"
users[new_user_addr] = {
    'name': new_user_name,
    'address': new_user_addr
}
# --- Explicitly add the new user to the contract ---
# This ensures the UI can find it, set it as the focus node, and read its initial (empty) state.
contract._get_or_create_idx(new_user_addr)
if new_user_addr not in contract.nodes:
        contract.nodes[new_user_addr] = Node()


print(f"\n📊 Network created:")
print(f"  - Users: {len(users)} (8 initial + 1 'User')")
print(f"  - Initial vouches: {contract.network.number_of_edges()}")
print(f"  - Seed vouches used: {contract.seed_vouch_count}")

### Step 2: Create the UI 

We will now create the User Interface object in the background. We won't display it just yet. This allows us to perform scripted actions in Step 3 and have them update the UI object's state.

The focus is on the node 'User'.

In [ ]:
import matplotlib.pyplot as plt

# Turn off interactive mode
plt.ioff()

# Create UI object but DO NOT display it yet. We will display it at the end.
# --- Set current user to our new 'User' node ---
# We retrieve the address from the variable defined in Cell 3
ui = SYBMvpUserInterface(contract=contract, users=users, current_user_address=new_user_addr)

print("UI object created. We will display it after the scripted tutorial.")
print(f"The UI will be focused on: {new_user_name} ({new_user_addr[:12]}...)")

### Initial Network State

Let's see the network after initialization, before we make any changes.

> NOTE: You cannot unvouch for anyone initially since you haven't vouched for anyone yet.

#### Current User Section

The **Current User** section displays information about the account you're currently using:

- **Name**: The alphabetical name assigned to this user (e.g., Alice, Bob, Charlie)
- **Address**: The Ethereum-style address of the current user account
- **Rank**: The user's rank in the network (lower is better, "DEFAULT" means unranked)
- **Score**: The computed score based on vouching relationships and network structure (higher is better)
- **Outdegree**: Number of users this account has vouched for
- **In-degree**: Number of users who have vouched for this account

#### Network Status Section

The **Network Status** section provides a comprehensive overview of the entire network:

- **Total Users**: The number of active accounts in the network
- **Total Vouches**: The total number of vouching relationships (edges) in the network
- **User Rankings Table**: A sortable table showing all users with their:
  - **User**: The alphabetical name of each user
  - **Rank**: Current rank (lower is better, "DEFAULT" means the nodes has just been initialized, and "0" means unranked)
  - **Prev Rank**: Previous rank value (shows "-" if no previous rank existed)
  - **Score**: Current computed score
  - **Prev Score**: Previous score value (shows 0 if no previous score existed)
  - **Out**: Number of outgoing vouches (who this user vouches for)
  - **In**: Number of incoming vouches (who vouches for this user)

The table is sorted by score (highest first), and cells are highlighted in yellow when rank or score has changed, making it easy to track network dynamics.

In [ ]:
# Now, we can display various network states using the plotting utilities.
# We import the plotting functions
from plot_utils import show_network_status, show_network_graph
# Show the initial states
show_network_status(contract, users, ui)
show_network_graph(contract, users, ui)

## Step 3: Scripting Network Actions
 
Let's walk through a few examples illustrating actions from the perspective of our main 'User'.
 
We'll have one of the original network members, 'Alice', vouch for our 'User' (who is new to the network).

In [ ]:
# Get Alice's address (the first user in the original 8)
alice_addr = list(users.keys())[0]
alice_name = ui._get_display_name(users[alice_addr]['name']) # This should be 'Alice'

# Get our 'User's address (the 9th user we added)
# 'new_user_addr' and 'new_user_name' are from Cell 3
user_addr = new_user_addr
user_name = new_user_name

# 1. Make Alice vouch for User
print(f"Action: {alice_name} is vouching for {user_name}...")
try:
    # This is the 'User's first interaction, adding them to the contract
    contract.vouch(alice_addr, user_addr)
    print("...Vouch successful! Displaying updated network state:")
 
    # Refresh the UI object (for Step 4)
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

### Receiving a Vouch

Now, let's have another user, 'Bob', also vouch for our 'User'. This will change the 'User''s rank and score.

In [ ]:
# Get Bob's address (the second user)
bob_addr = list(users.keys())[1]
bob_name = ui._get_display_name(users[bob_addr]['name']) # This should be 'Bob'

# Get our 'User's address
user_addr = new_user_addr
user_name = new_user_name

# 2. Make Bob vouch for User
print(f"Action: {bob_name} is vouching for {user_name}...")
try:
    contract.vouch(bob_addr, user_addr)
    print("...Vouch successful! Displaying updated network state:")
 
    # Refresh the UI object
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

### Vouching Back

A vouch can also be reciprocated. Let's have our 'User' vouch back for 'Alice'.

In [ ]:
# Get our 'User's address
user_addr = new_user_addr
user_name = new_user_name

# Get Alice's address
alice_addr = list(users.keys())[0]
alice_name = ui._get_display_name(users[alice_addr]['name'])

# 3. Make User vouch for Alice
print(f"Action: {user_name} is vouching for {alice_name}...")
try:
    contract.vouch(user_addr, alice_addr)
    print("...Vouch successful! Displaying updated network state:")
 
    # Refresh the UI object
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

### Unvouching

Relationships can change. Let's say 'Alice' decides to unvouch for our 'User'.

In [ ]:
# Get Alice's address
alice_addr = list(users.keys())[0]
alice_name = ui._get_display_name(users[alice_addr]['name'])

# Get our 'User's address
user_addr = new_user_addr
user_name = new_user_name

# 4. Make Alice unvouch for User
print(f"Action: {alice_name} is unvouching for {user_name}...")
try:
    contract.unvouch(alice_addr, user_addr)
    print("...Unvouch successful! Displaying updated network state:")
 
    # Refresh the UI object
    ui.vouch_target.options = ui._get_vouchable_users()
    ui.unvouch_target.options = ui._get_unvouchable_users()
    ui._update_all()
    
    # Show the current state
    show_network_status(contract, users, ui)
    show_network_graph(contract, users, ui)
except Exception as e:
    print(f"Vouch failed: {e}")

## Step 4: Launch Interactive UI

Now that we have programmatically guided you through the main operations, let's display the UI.

You can now freely interact with the network in its current state. The UI below reflects all the changes we just scripted in Step 3.

In [ ]:
print("\n" + "=" * 60)
print("Displaying UI...")
print("=" * 60)

ui.display()